In [1]:
from pandas import DataFrame, Series
import numpy as np

In [2]:
import invisible_cities.reco.peak_functions_c as cpf
import invisible_cities.reco.pmaps_functions_c as cpm

### Create S12 df

In [3]:
def s12_dataframe():
    evs  = [   0,     0,     0,     0,     0,      3,     3]
    peak = [   0,     0,     1,     1,     1,      0,     0]
    time = [1000., 1025., 2050., 2075., 2100., 5000., 5025.]
    ene  = [123.4, 140.8, 730.0, 734.2, 732.7, 400.0, 420.3]
    df = DataFrame.from_dict(dict(
            event  = Series(evs , dtype=np.  int32),
            evtDaq = evs,
            peak   = Series(peak, dtype=np.   int8),
            time   = Series(time, dtype=np.float32),
            ene    = Series(ene , dtype=np.float32),
        ))
    return df



In [4]:
dfs2 = s12_dataframe()

In [5]:
dfs2

,ene,event,evtDaq,peak,time
0,123.400002,0,0,0,1000.0
1,140.800003,0,0,0,1025.0
2,730.000000,0,0,1,2050.0
3,734.200012,0,0,1,2075.0
4,732.700012,0,0,1,2100.0
5,400.000000,3,3,0,5000.0
6,420.299988,3,3,0,5025.0


### pmaps functions

In [6]:
s1_dict = cpm.df_to_s1_dict(dfs2)

In [7]:
s1_dict

{0: S1 (number of peaks = 2)
 peak number = 0: Peak(samples = 2 width =      0.0 mus , energy =    264.2 pes
         height =    140.8 pes tmin-tmax = minmax(min=1.0, max=1.025) mus  
 peak number = 1: Peak(samples = 3 width =      0.1 mus , energy =   2196.9 pes
         height =    734.2 pes tmin-tmax = minmax(min=2.05, max=2.1) mus  ,
 3: S1 (number of peaks = 1)
 peak number = 0: Peak(samples = 2 width =      0.0 mus , energy =    820.3 pes
         height =    420.3 pes tmin-tmax = minmax(min=5.0, max=5.025) mus  }

In [8]:
s1_dict[0].s1d

{0: [array([ 1000.,  1025.]), array([ 123.40000153,  140.80000305])],
 1: [array([ 2050.,  2075.,  2100.]),
  array([ 730.        ,  734.20001221,  732.70001221])]}

In [9]:
s1_dict[0].peaks

{0: Peak(samples = 2 width =      0.0 mus , energy =    264.2 pes
         height =    140.8 pes tmin-tmax = minmax(min=1.0, max=1.025) mus ,
 1: Peak(samples = 3 width =      0.1 mus , energy =   2196.9 pes
         height =    734.2 pes tmin-tmax = minmax(min=2.05, max=2.1) mus }

In [10]:
def dump_s12_dict(s12_dict):
    for event_no, s12 in s12_dict.items():
        print('event number = {}'.format(event_no))
        print('peaks in s1/s2 = {}'.format(s12.peak_collection()))
        for peak_no in s12.peak_collection():
            print('peak number = {}'.format(peak_no))
            pwf = s12.peak_waveform(peak_no)
            print('Waveform: time = {}, E = {}'.format(pwf.t, pwf.E))

In [11]:
def dump_s2si_dict(s2si_dict):
    for event_no, si in s2si_dict.items():
        print('event number = {}'.format(event_no))
        si = s2si_dict[event_no]
        for peak_no in si.peak_collection():
            print('peak number = {}'.format(peak_no))
            peak = si.peak_waveform(peak_no)
            print("S2: t = {}".format(peak.t))
            print("S2: E = {}".format(peak.E))
            
            for sipm_no in si.sipms_in_peak(peak_no):
                print("sipm number = {}".format(sipm_no))
                sipm_wfm = si.sipm_waveform(peak_no, sipm_no)
                print("Si: t = {}".format(sipm_wfm.t))
                print("Si: E = {}".format(sipm_wfm.E))
                

In [12]:
dump_s12_dict(s1_dict)

event number = 0
peaks in s1/s2 = (0, 1)
peak number = 0
Waveform: time = [ 1000.  1025.], E = [ 123.40000153  140.80000305]
peak number = 1
Waveform: time = [ 2050.  2075.  2100.], E = [ 730.          734.20001221  732.70001221]
event number = 3
peaks in s1/s2 = (0,)
peak number = 0
Waveform: time = [ 5000.  5025.], E = [ 400.          420.29998779]


In [13]:
s2_dict = cpm.df_to_s2_dict(dfs2)

In [14]:
dump_s12_dict(s2_dict)

event number = 0
peaks in s1/s2 = (0, 1)
peak number = 0
Waveform: time = [ 1000.  1025.], E = [ 123.40000153  140.80000305]
peak number = 1
Waveform: time = [ 2050.  2075.  2100.], E = [ 730.          734.20001221  732.70001221]
event number = 3
peaks in s1/s2 = (0,)
peak number = 0
Waveform: time = [ 5000.  5025.], E = [ 400.          420.29998779]


In [15]:
s2_dict[0].peaks[0].good_waveform

True

### Create S2Si DF

In [16]:
def s2si_dataframe():
    evs  = [  0,   0,  0,  0,   0,  0,  0,  3,  3,  3,  3]
    peak = [  0,   0,  0,  0,   1,  1,  1,  0,  0,  0,  0]
    sipm = [  0,   0,  1,  1,   3,  3,  3, 10, 10, 15, 15]
    ene  = [1.5, 2.5, 15, 25, 5.5, 10, 20,  8,  9, 17, 18]

    dfs2si = DataFrame.from_dict(dict(
        event   = Series(evs , dtype=np.int32),
        evtDaq  = evs,
        peak    = Series(peak, dtype=np.   int8),
        nsipm   = Series(sipm, dtype=np.  int16),
        ene     = Series(ene , dtype=np.float32),
    ))
    return dfs2si

In [17]:
dfsi = s2si_dataframe()

In [18]:
dfsi

,ene,event,evtDaq,nsipm,peak
0,1.5,0,0,0,0
1,2.5,0,0,0,0
2,15.0,0,0,1,0
3,25.0,0,0,1,0
4,5.5,0,0,3,1
5,10.0,0,0,3,1
6,20.0,0,0,3,1
7,8.0,3,3,10,0
8,9.0,3,3,10,0
9,17.0,3,3,15,0


In [19]:
s2si_dict = cpm.df_to_s2si_dict(dfs2, dfsi)

In [20]:
print(s2si_dict)

{0: ================================================================================
S2 (number of peaks = 2)
peak number = 0: Peak(samples = 2 width =      0.0 mus , energy =    264.2 pes
        height =    140.8 pes tmin-tmax = minmax(min=1.0, max=1.025) mus  
peak number = 1: Peak(samples = 3 width =      0.1 mus , energy =   2196.9 pes
        height =    734.2 pes tmin-tmax = minmax(min=2.05, max=2.1) mus  
--------------------------------------------------------------------------------
SiPMs for non-empty peaks

peak number = 0: nsipm in peak = (0, 1)

peak number = 1: nsipm in peak = (3,)
--------------------------------------------------------------------------------
SiPMs Waveforms

peak number = 0: sipm number = 0
    sipm waveform (zs) = Peak(samples = 2 width =      0.0 mus , energy =      4.0 pes
        height =      2.5 pes tmin-tmax = minmax(min=1.0, max=1.025) mus 
peak number = 0: sipm number = 1
    sipm waveform (zs) = Peak(samples = 2 width =      0.0 mus , energy

In [21]:
dump_s2si_dict(s2si_dict)

event number = 0
peak number = 0
S2: t = [ 1000.  1025.]
S2: E = [ 123.40000153  140.80000305]
sipm number = 0
Si: t = [ 1000.  1025.]
Si: E = [ 1.5  2.5]
sipm number = 1
Si: t = [ 1000.  1025.]
Si: E = [ 15.  25.]
peak number = 1
S2: t = [ 2050.  2075.  2100.]
S2: E = [ 730.          734.20001221  732.70001221]
sipm number = 3
Si: t = [ 2050.  2075.  2100.]
Si: E = [  5.5  10.   20. ]
event number = 3
peak number = 0
S2: t = [ 5000.  5025.]
S2: E = [ 400.          420.29998779]
sipm number = 10
Si: t = [ 5000.  5025.]
Si: E = [ 8.  9.]
sipm number = 15
Si: t = [ 5000.  5025.]
Si: E = [ 17.  18.]
